# Step 1: Prepapre documents

In [1]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

Two importan concepts in Elasticsearch:
1. Documents, collection of fields with associated values and add the documents to an index.
2. Indexes, collection of documents that is stored in a highly optimized format designed to perform efficient searches.

In [2]:
documents_raw

[{'course': 'data-engineering-zoomcamp',
  'documents': [{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?'},
   {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
    'section': 'General course-related questions',
    'question': 'Course - What are the prerequisites for this course?'},
   {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in

In [3]:
documents = []

for course_dict in documents_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

# Step 2: Create Embeddings using Pretrained Models

`all-mpnet-base-v2`

In [3]:
from sentence_transformers import SentenceTransformer

/home/ubuntu/.local/share/virtualenvs/llm-zoomcamp-2024-rEyYMrOL/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
model  = SentenceTransformer('all-mpnet-base-v2')

In [7]:
model.encode("This is a simple sentence")

array([ 4.44871606e-03, -7.61314183e-02, -3.77412420e-04,  7.52521865e-03,
       -3.80979255e-02,  3.80130820e-02, -9.73012205e-03, -5.05396351e-03,
       -9.37976222e-03,  1.23888915e-02,  4.91276830e-02,  1.52209224e-02,
        3.80008072e-02, -6.41803369e-02,  9.42136627e-03, -5.19748107e-02,
        9.08066407e-02,  1.71114560e-02,  1.62125099e-02,  2.98867375e-02,
        1.50533975e-03,  8.35077651e-03,  3.78842764e-02, -1.01193022e-02,
        6.46110578e-03,  3.97833392e-05, -1.45217869e-02, -1.88468881e-02,
       -3.74040045e-02, -1.51656172e-03, -1.02681359e-02, -3.68063822e-02,
        2.36677490e-02, -6.46020845e-02,  1.96967062e-06, -5.01107937e-03,
       -2.80826306e-03, -1.92073397e-02, -8.65120068e-02,  2.83463895e-02,
       -5.38666546e-02,  3.63704786e-02, -2.26467345e-02,  2.87367441e-02,
       -1.32342288e-02,  1.08689621e-01,  3.70519236e-02,  3.38801853e-02,
       -5.30679710e-02,  3.61782461e-02, -1.35720137e-03, -3.63482796e-02,
       -2.78346594e-02, -

In [8]:
len(model.encode("This is a simple sentence"))

768

In [9]:
# created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # operations.appand({'index': {'_index': 'doc_ix'}})
    # transforming the title into an embedding using the model
    doc['text_vector'] = model.encode(doc['text']).tolist()
    operations.append(doc)

# Step 3: Set Up ElasticSearch Connection

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

es_client.info()

ObjectApiResponse({'name': 'a7a4e93b8b29', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'FbPLjy97Rw-YH-qSuQFV9Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Steop 4: Create Mappings and Index

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {"type":"dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [10]:
index_name = "course-question"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

# Step 5: Add documents into index

In [14]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Step 6: Create end user query

In [5]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [6]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [11]:
res = es_client.search(index=index_name, knn=query, source=['text', 'section', 'question', 'course'])

In [12]:
res['hits']['hits']

[{'_index': 'course-question',
  '_id': '0jCGt5ABUb935UUv5_0S',
  '_score': 0.7147918,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-question',
  '_id': '5TGHt5ABUb935UUvBwCl',
  '_score': 0.6134736,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at o

# Step 7: Perform Semantic Search and Advanced Search

In [14]:
response = es_client.search(
    index=index_name,
    query={
        "bool": {
            "must": {
                "multi_match":{
                    "query": "windows or python?",
                    "fields": ["text", "question", "course", "title"],
                    "type": "best_fields"
                },
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
)

In [16]:
response['hits']['hits']

[{'_index': 'course-question',
  '_id': 'GDCGt5ABUb935UUv6v5-',
  '_score': 7.728908,
  '_source': {'text': 'Problem: If you have already installed pgcli but bash doesn\'t recognize pgcli\nOn Git bash: bash: pgcli: command not found\nOn Windows Terminal: pgcli: The term \'pgcli\' is not recognized…\nSolution: Try adding a Python path C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts to Windows PATH\nFor details:\nGet the location: pip list -v\nCopy C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\site-packages\n3. Replace site-packages with Scripts: C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts\nIt can also be that you have Python installed elsewhere.\nFor me it was under c:\\python310\\lib\\site-packages\nSo I had to add c:\\python310\\lib\\Scripts to PATH, as shown below.\nPut the above path in "Path" (or "PATH") in System Variables\nReference: https://stackoverflow.com/a/68233660',
   'section': 'Module 1: Docker and Terraform',
   'question': 'PGCLI - pgc

Query above is **keyword search**.

In [20]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
            "course": "data-engineering-zoomcamp"
            }
    },
    knn = knn_query,
    size=5,
    explain=True
)

In [21]:
response['hits']['hits']

[{'_shard': '[course-question][0]',
  '_node': '391UgMY7Tbq4HWH5fyd0vw',
  '_index': 'course-question',
  '_id': '0jCGt5ABUb935UUv5_0S',
  '_score': 1.4937057,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.02696547843515873,
    -0.0006259932997636497,
    -0.016629500314593315,
    0.052851494401693344,
    0.05476532131433487,
    -0.03133988752961159,
    0.029942717403173447,
    -0.04808562621474266,
    0.04467545449733734,
    0.005839466117322445,
    0.016233088448643684,
    0.012001161463558674,
    -0.03122236765921116,
    0.0166005901992321,
    -0.04886902868747711,
    -0.06496307998895645,
    0.0464341901242733,
    -0.009297674521803856,
    -0.06425288319587708,
    -0.013732724823057652,
   

Semantic search -> Search by the underlying meaning (by vector from embeddings)